## 디스코드에서 공유한 `.env` 파일을 sesac-project/data/practice (지금 이 폴더)에 집어 넣기

In [1]:
!source .venv/Scripts/activate
# !source ../../.venv/bin/activate
!pip install python-dotenv openai

'source'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/223.4 kB ? eta -:--:--
   --------------------------------------- 223.4/223.4 kB 14.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.9 kB ? eta -:--:--
   ---------------------------------------- 75.9/75.9 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.9 kB ? eta -:--:--
   ---------------------------------------- 76.9/76.9 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 78.3/78.3 kB 4.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
import openai
client = OpenAI()
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [9]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.7):
    completion = client.chat.completions.create(
    model = model,
    messages=[
        {"role": "user", "content": prompt}
    ],
    temperature=temperature
    )
    return completion.choices[0].message.content

## 1. 프롬프팅의 대원칙

- **원칙 1: 명확하고 구체적인 지시사항을 작성하자**
- **원칙 2: 모델이 "생각"할 시간을 주자**


- 아래 실습은 [Andrew Ng이 만든 프롬프트 수업](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/1/introduction) 의 실습 코드를 번역한 것입니다.

### 명확하고 구체적인 지시사항을 작성하자.

#### 전략 1: 입력의 구분된 부분을 명확하게 나타내기 위해 구분자 사용

- 구분자는 ```, """, < >, `<tag> </tag>`, `:` 등이 될 수 있습니다.


In [10]:
# 구분자를 사용하면 원하는 응답을 얻을 가능성이 높아진다. ```{text}```와 같이.
text = f"""
모델에게 원하는 작업을 수행하도록 지시하는 것은 가능한 한 명확하고 구체적인 지시를 제공함으로써 \
이루어집니다. 이것은 모델을 원하는 출력 방향으로 이끌고, 관련 없거나 잘못된 응답을 받을 가능성을 \
줄입니다. 명확한 프롬프트 작성을 간단한 프롬프트 작성과 혼동하지 마십시오. 많은 경우, 더 긴 프롬프트는 \
모델에게 더 많은 명확성과 맥락을 제공하여 \
더 자세하고 관련 있는 출력을 이끌어낼 수 있습니다.
"""
prompt = f"""
세 개의 백틱으로 구분된 텍스트를 \
하나의 문장으로 요약하십시오.
```{text}```
"""
response = get_completion(prompt)
print(response)

모델에게는 명확하고 구체적인 지시를 제공하여 원하는 작업을 수행하게 하는 것이 중요합니다. 짧은 프롬프트와 긴 프롬프트를 혼동하지 말고, 더 많은 명확성과 맥락을 제공하는 긴 프롬프트를 사용하는 것이 좋습니다.


#### 전략 2: 구조화된 출력 요청

- JSON, HTML

In [11]:
# JSON 형식으로 제공해 달라는 언급이 없으면 이상한 답을 뱉을 확률이 높아진다.
prompt = f"""
세 권의 가상의 책 제목과 그 저자, 장르를 생성하세요. \
book_id, title, author, genre라는 키를 포함하여 \
JSON 형식으로 제공해 주세요.
"""
response = get_completion(prompt)
print(response)

{
  "book1": {
    "title": "The Secret Garden",
    "author": "Frances Hodgson Burnett",
    "genre": "Children's literature"
  },
  "book2": {
    "title": "The Great Gatsby",
    "author": "F. Scott Fitzgerald",
    "genre": "Classic literature"
  },
  "book3": {
    "title": "The Hunger Games",
    "author": "Suzanne Collins",
    "genre": "Young adult fiction"
  }
}


#### 전략 3: 조건 충족 여부 확인 요청

In [12]:
text_1 = f"""
차를 만드는 것은 쉽습니다! 먼저, \
끓는 물을 준비해야 합니다. 그 동안, \
컵을 잡고 그 안에 차백을 넣으세요. 물이 \
충분히 뜨거워지면, 그냥 차백 위에 붓습니다. \
차가 우러나기를 잠시 기다린 후, \
몇 분 후에 차백을 꺼냅니다. 원하신다면, \
맛에 따라 설탕이나 우유를 추가할 수 있습니다. \
그게 다입니다! 맛있는 차 한 잔을 즐길 수 있습니다.
"""
prompt = f"""
세 개의 쌍따옴표로 구분된 텍스트가 제공됩니다. \
그것이 일련의 지시사항을 포함하고 있다면, \
다음 형식으로 해당 지시사항을 다시 작성하십시오:

단계 1 - ...
단계 2 - …
…
단계 N - …

텍스트에 일련의 지시사항이 포함되어 있지 않다면, \
단순히 \"단계가 제공되지 않음\"이라고 작성하십시오.

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1에 대한 완성:")
print(response)

Text 1에 대한 완성:
단계 1 - 끓는 물을 준비합니다.
단계 2 - 컵을 잡고 그 안에 차백을 넣습니다.
단계 3 - 물이 충분히 뜨거워지면, 차백 위에 물을 붓습니다.
단계 4 - 차가 우러나기를 잠시 기다립니다.
단계 5 - 몇 분 후에 차백을 꺼냅니다.
단계 6 - 원하신다면, 맛에 따라 설탕이나 우유를 추가할 수 있습니다.
단계 7 - 맛있는 차 한 잔을 즐길 수 있습니다.


In [13]:
text_2 = f"""
오늘은 햇살이 밝게 빛나고, 새들이 \
노래하고 있습니다. 공원에서 산책하기에 \
아름다운 날입니다. 꽃들이 피고, 나무들이 \
산들바람에 부드럽게 흔들리고 있습니다. 사람들은 \
아름다운 날씨를 즐기기 위해 밖에 나와 있습니다. \
일부는 소풍을 즐기고, 다른 일부는 게임을 하거나 \
잔디밭에서 그냥 휴식을 취하고 있습니다. 자연의 \
아름다움을 감상하며 야외에서 시간을 보내기에 \
완벽한 날입니다.
"""
prompt = f"""
세 개의 쌍따옴표로 구분된 텍스트가 제공됩니다. \
그것이 일련의 지시사항을 포함하고 있다면, \
다음 형식으로 해당 지시사항을 다시 작성하십시오:

단계 1 - ...
단계 2 - …
…
단계 N - …

텍스트에 일련의 지시사항이 포함되어 있지 않다면, \
단순히 \"단계가 제공되지 않음\"이라고 작성하십시오.

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2에 대한 완성:")
print(response)

Text 2에 대한 완성:
단계가 제공되지 않음


## 실습하기

- "뜨개질 하는 법"을 검색해서 전략 3에서 본 지시사항 다시 작성하기 테스크를 위한 프롬프트를 작성하세요. 

- 이 때 조건 충족여부를 체크하는 프롬프트를 꼭 포함하세요.

In [22]:
text_3="""
뜨개질을 하려면 우선 재료를 모은다.  \
먼저 첫 번째 색상으로 12줄을 뜬다. \
이제 12줄을 다 뜨고 나면, 가위로 실을 자른다. 약 15cm정도 남기고 잘라야 한다. \
두 번째 색상의 실을 뜨기 시작한다. 5코를 뜨고 멈춰서 끝을 당긴다. \
이때 끝은 묶지 않고 둔다. 나중에 태피스트리 바늘이나 코바늘로 엮을 것이다. \
새로운 실로 12줄을 뜨고 나면, 세 번째 실을 더한다. \
이제 뜨개질을 마무리한다. 목도리를 목에 감싸서 자신의 작품을 감상한다. 기분이 좋지 않은가?
"""
prompt=f'''
세 개의 쌍따옴표로 구분된 텍스트를 제공해 드릴게요. \
이 텍스트가 일련의 지시사항을 포함하고 있다면, \
다음 형식으로 해당 지시사항을 다시 작성해 주세요:

단계 1 - ...
단계 2 - …
…
단계 N - …

텍스트에 일련의 지시사항이 포함되어 있지 않다면, \
\"단계가 제공되지 않음\"이라고 작성해 주세요.

\"\"\"{text_3}\"\"\"
'''
response = get_completion(prompt)
print("실습에 대한 완성:")
print(response)

실습에 대한 완성:
단계 1 - 재료를 모으세요.
단계 2 - 첫 번째 색상으로 12줄을 뜨세요.
단계 3 - 12줄을 다 뜨고 나면, 가위로 실을 자르세요. 약 15cm를 남기고 잘라야 합니다.
단계 4 - 두 번째 색상의 실을 뜨기 시작하세요. 5코를 뜨고 멈추고, 끝을 당기세요. 이때 끝은 묶지 않고 둬야 합니다. 나중에 태피스트리 바늘이나 코바늘로 엮을 것입니다.
단계 5 - 새로운 실로 12줄을 뜨고 나면, 세 번째 실을 더하세요.
단계 6 - 뜨개질을 마무리하세요.
단계 7 - 목도리를 목에 감싸서 자신의 작품을 감상하세요. 기분이 좋지 않으신가요?


#### 전략 4: "Few-shot" 프롬프팅

In [24]:
prompt = f"""
일관된 스타일로 답변하는 것이 과제입니다.

<아이>: 인내에 대해 가르쳐 주세요.

<할머니/할아버지>: 가장 깊은 계곡을 파는 강은 \
겸손한 봄에서 흐른단다; 가장 웅장한 교향곡은 \
하나의 단일한 음에서 시작되지; 가장 복잡한 태피스트리는 \
단 하나의 실로부터 시작되는 것이야.

<아이>: 회복력에 대해 가르쳐 주세요.
"""
response = get_completion(prompt)
print(response)

<할머니/할아버지>: 나무는 일단 쓰러지면 다시 일어서기 전에 잠시 쉬어야 해; 강은 장애물을 만나면 그 주위로 흐르기 시작해; 사람은 어려운 시기를 겪으면 그 경험을 바탕으로 더욱 강해져야 해.



### 원칙 2: 모델에게 "생각"할 시간을 주자

#### 전략 1: 작업을 완료하기 위해 필요한 단계 명시

In [25]:
text = f"""
매력적인 마을에서, 잭과 질이라는 형제는 \
언덕 꼭대기에 있는 우물에서 물을 길으러 가는 \
모험을 떠납니다. 그들이 즐겁게 노래를 부르며 올라가던 중, \
불행이 닥쳤습니다—잭이 돌에 걸려 넘어져 \
언덕 아래로 굴러떨어졌고, 질도 뒤따라 넘어졌습니다. \
약간 다쳤지만, 그들은 위로하는 포옹 속으로 \
집으로 돌아왔습니다. 사건에도 불구하고, \
그들의 모험심은 여전히 빛났고, 그들은 기쁨으로 계속 탐험을 이어갔습니다.
"""
# 예시 1
prompt_1 = f"""
다음 작업을 수행하십시오:
1 - 세 개의 백틱(```)으로 구분된 다음 텍스트를 한 문장으로 요약합니다.
2 - 요약문을 프랑스어로 번역합니다.
3 - 프랑스어 요약문에 있는 각 이름을 나열합니다.
4 - 다음 키를 포함하는 json 객체를 출력합니다: french_summary, num_names.

각 답변을 줄바꿈으로 구분하십시오.

텍스트:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1에 대한 완성:")
print(response)

prompt 1에 대한 완성:
1 - In a charming village, brothers Jack and Jill embark on an adventure to fetch water from a well on top of a hill. While joyfully singing their way up, tragedy strikes—Jack trips on a rock and tumbles down the hill, followed by Jill. Despite some injuries, they embrace each other and return home. Undeterred by the incident, their sense of adventure still shines, and they continue exploring with delight.

2 - Dans un charmant village, les frères Jack et Jill se lancent dans une aventure pour aller chercher de l'eau d'un puits au sommet d'une colline. Tout en chantant joyeusement en montant, la tragédie frappe - Jack trébuche sur un rocher et dévale la colline, suivi de Jill. Malgré quelques blessures, ils se serrent dans les bras l'un de l'autre et rentrent chez eux. Indifférents à l'incident, leur sens de l'aventure brille toujours, et ils continuent d'explorer avec délice.

3 - Jack, Jill

4 - {
  "french_summary": "Dans un charmant village, les frères Jack et Jill

#### 지정된 형식으로 출력 요청

In [26]:
prompt_2 = f"""
다음 작업을 수행하는 것이 과제입니다:
1 - <>로 구분된 다음 텍스트를 한 문장으로 요약합니다.
2 - 요약문을 프랑스어로 번역합니다.
3 - 프랑스어 요약문에 있는 각 이름을 나열합니다.
4 - 다음 키를 포함하는 json 객체를 출력합니다: french_summary, num_names.

다음 형식을 사용하십시오:
텍스트: <요약할 텍스트>
요약: <요약문>
번역: <요약문 번역>
이름: <요약문 내 이름 목록>
JSON 출력: <요약과 num_names가 있는 json>

텍스트: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2에 대한 완성:")
print(response)


prompt 2에 대한 완성:
요약: 잭과 질은 매력적인 마을에서 우물에서 물을 길으러 가는 모험을 떠납니다. 그러나 잭이 굴러떨어지고 질도 넘어져 다쳤지만, 그들은 모험심을 잃지 않고 계속 탐험을 이어갑니다.
번역: Jack et Jill partent à l'aventure pour aller chercher de l'eau dans un puits au sommet d'une colline dans un charmant village. Cependant, Jack tombe et roule en bas de la colline, entraînant Jill avec lui. Bien qu'ils soient légèrement blessés, ils retournent chez eux avec un câlin réconfortant. Malgré cet incident, leur esprit d'aventure brille toujours et ils continuent leur exploration avec joie.
이름: Jack, Jill
JSON 출력: {"french_summary": "Jack et Jill partent à l'aventure pour aller chercher de l'eau dans un puits au sommet d'une colline dans un charmant village. Cependant, Jack tombe et roule en bas de la colline, entraînant Jill avec lui. Bien qu'ils soient légèrement blessés, ils retournent chez eux avec un câlin réconfortant. Malgré cet incident, leur esprit d'aventure brille toujours et ils continuent leur exploration avec joie.", "num_names": 2}


#### 전략 2: 모델이 결론을 서둘러서 도달하기 전에 자체 해결책을 찾도록 지시한다.

In [27]:
prompt = f"""
학생의 해결책이 올바른지 아닌지 판단하십시오.

질문:
저는 태양광 발전 설치를 구축하고 있고, 재정 계산에 \
도움이 필요합니다.
- 땅은 평방 피트당 $100입니다.
- 태양광 패널을 평방 피트당 $250에 구입할 수 있습니다.
- 저는 유지보수에 대한 계약을 협상하여 연간 고정 $100k, \
추가로 평방 피트당 $10의 비용이 듭니다.
제곱피트 수에 따른 첫 해 운영 총비용은 얼마입니까?

학생의 해결책:
설치 크기를 평방 피트 단위의 x라고 합시다.
비용:
1. 땅 비용: 100x
2. 태양광 패널 비용: 250x
3. 유지보수 비용: 100,000 + 100x
총비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

학생의 해결책은 올바릅니다. 학생은 설치 크기를 x로 가정하고, 각 항목의 비용을 정확히 계산하였습니다. 따라서 첫 해 운영 총비용은 450x + 100,000입니다.


#### 실제로 학생의 해결책은 올바르지 않다.

#### 모델에게 먼저 자체 해결책을 작업하도록 지시함으로써 이를 수정할 수 있습니다.

In [28]:
prompt = f"""
학생의 해결책이 올바른지 아닌지 판단하는 것이 과제입니다.
문제를 해결하기 위해 다음을 수행하십시오:
- 먼저, 문제에 대한 자신만의 해결책을 포함하여 최종 총계를 계산합니다.
- 그런 다음 학생의 해결책과 비교하여 학생의 해결책이 올바른지 평가합니다.
문제를 스스로 풀어보기 전까지 학생의 해결책이 올바른지 결정하지 마십시오.

다음 형식을 사용하십시오:
질문:
```
여기에 질문
```
학생의 해결책:
```
여기에 학생의 해결책 
```
실제 해결책:
```
여기에 해결책을 작업하는 단계와 당신의 해결책 
```
학생의 해결책이 방금 계산한 실제 해결책과 같은지:
```
예 또는 아니오
```
학생 성적:
```
정확함 또는 부정확함
```
질문:
```
태양광 발전 설치를 구축하고 있으며 재정적 계산에 도움이 필요합니다.
- 땅은 평방 피트당 $100입니다.
- 태양광 패널은 평방 피트당 $250에 구입할 수 있습니다.
- 유지보수 계약을 체결하여 연간 고정 $100k의 비용이 들며, 추가로 평방 피트당 $10이 추가됩니다.
제곱피트 수에 따른 첫 해 운영 총비용은 얼마입니까?
```
학생의 해결책:
```
설치 크기를 평방 피트 단위의 x라고 합시다.
비용:
1. 땅 비용: 100x
2. 태양광 패널 비용: 250x
3. 유지보수 비용: 100,000 + 100x
총비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
실제 해결책:
"""
response = get_completion(prompt)
print(response)

설치 크기를 평방 피트 단위의 x라고 합시다.
비용:
1. 땅 비용: 100x
2. 태양광 패널 비용: 250x
3. 유지보수 비용: 100,000 + 10x
총비용: 100x + 250x + 100,000 + 10x = 360x + 100,000



## 모델의 한계: 환각

- Boie는 실제 회사이지만, 제품명은 실제가 아닙니다.

In [29]:
prompt = f"""
Boie의 AeroGlide UltraSlim Smart Toothbrush에 대해 알려주세요.
"""
response = get_completion(prompt)
print(response)

Boie의 AeroGlide UltraSlim Smart Toothbrush는 혁신적인 디자인과 최신 기술을 통해 탁월한 구강 위생을 제공하는 스마트 칫솔입니다.

AeroGlide UltraSlim Smart Toothbrush는 초슬림하고 가벼운 디자인으로 제작되어 손쉽게 사용할 수 있습니다. 칫솔의 미세한 진동과 부드러운 섬유로 치아와 잇몸을 깨끗하게 청소하면서도 부드럽게 마사지해줍니다.

이 칫솔은 스마트 기능을 갖추고 있어 사용자의 구강 위생 관리를 도와줍니다. 칫솔의 손잡이에는 블루투스 기술이 내장되어 있어 스마트폰과 연동하여 실시간으로 구강 청결 상태를 모니터링할 수 있습니다. 앱을 통해 칫솔 사용 시간, 압력, 세척 횟수 등의 정보를 확인할 수 있어 구강 위생 관리에 도움을 줍니다.

AeroGlide UltraSlim Smart Toothbrush는 사용자의 개인적인 선호에 따라 세 가지 모드(청결, 민감, 마사지)를 지원합니다. 이렇게 다양한 모드를 통해 사용자는 자신에게 가장 적합한 칫솔링 모드를 선택할 수 있습니다.

또한 칫솔은 USB로 충전할 수 있어 휴대하기에도 편리합니다. 충전한 후에는 약 30일 동안 사용할 수 있습니다. 또한 칫솔의 머리는 교체가 가능하며, 교체 주기는 약 3개월입니다.

Boie의 AeroGlide UltraSlim Smart Toothbrush는 혁신적인 디자인과 스마트 기능을 통해 사용자에게 탁월한 구강 위생을 제공합니다.


## 실습: 환각 문제를 해결하기 위해 어떻게 해야 할까요? 프롬프트로 작성해 보세요.


In [34]:
# YOUR CODE HERE.
prompt = f"""
내가 물어보는 이야기는 (```) 안에 들어 있어. 만약 네가 모르는 이야기 혹은 사건이면, 모른다고 대답해 줘.
```Boie의 AeroGlide UltraSlim Smart Toothbrush에 대해 알려주세요.```
"""
response = get_completion(prompt)
print(response)

죄송하지만, Boie의 AeroGlide UltraSlim Smart Toothbrush에 대해서는 제가 알려드릴 수 없습니다.


#### 백슬래시에 대한 참고사항

- 이 과정에서는 텍스트가 화면에 맞게 표시되도록 하기 위해 백슬래시 `\`를 사용하고 있습니다. 이는 새 줄 '\n' 문자를 삽입하지 않기 위함입니다.
- GPT-3는 실제로 줄바꿈 문자를 삽입하든 말든 크게 영향을 받지 않습니다. 하지만 일반적으로 LLM을 사용할 때 프롬프트의 줄바꿈 문자가 모델의 성능에 영향을 미칠 수 있습니다.

## 2. 반복적 프롬프트 작성

## 마케팅 제품 설명 생성하기: 제품 사양서를 기반으로

In [35]:
fact_sheet_chair = """
개요
- 중세 현대적인 사무용 가구의 아름다운 가족 중 하나로,
  파일 캐비닛, 책상, 책장, 회의용 테이블 등이 포함됩니다.
- 쉘 색상과 베이스 마감 옵션이 다양합니다.
- 플라스틱 뒷면과 앞면의 패브릭 장식이 있는 옵션(SWC-100)과
  10가지 패브릭 및 6가지 가죽 옵션이 있는 전체 장식 옵션(SWC-110)이 있습니다.
- 베이스 마감 옵션에는 스테인리스 스틸, 매트 블랙,
  광택 화이트, 크롬이 있습니다.
- 의자는 팔걸이가 있는 버전과 없는 버전이 있습니다.
- 가정이나 사업장에 적합합니다.
- 계약용으로 인증되었습니다.

구조
- 5개 바퀴가 있는 플라스틱 코팅 알루미늄 베이스.
- 손쉬운 상하 조정을 위한 공압 의자 조절 기능.

치수
- 폭 53 CM | 20.87”
- 깊이 51 CM | 20.08”
- 높이 80 CM | 31.50”
- 좌석 높이 44 CM | 17.32”
- 좌석 깊이 41 CM | 16.14”

옵션
- 부드러운 바닥 또는 단단한 바닥 캐스터 옵션.
- 좌석 폼 밀도 두 가지 선택:
  중간 (1.8 lb/ft3) 또는 높음 (2.8 lb/ft3)
- 팔걸이 없거나 8가지 위치 조정이 가능한 PU 팔걸이

재료
쉘 베이스 글라이더
- 수정된 나일론 PA6/PA66 코팅이 있는 주조 알루미늄.
- 쉘 두께: 10 mm.
좌석
- HD36 폼

제조국
- 이탈리아
"""
prompt = f"""
마케팅 팀을 도와 기술 사양서를 기반으로 소매 웹사이트에 대한 제품 설명을 만듭니다.

세 개의 백틱으로 구분된 기술 사양에 제공된 정보를 기반으로 제품 설명을 작성하세요.

기술 사양: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

이 제품은 중세 현대적인 사무용 가구 시리즈의 일부로, 파일 캐비닛, 책상, 책장, 회의용 테이블 등이 포함되어 있습니다. 다양한 쉘 색상과 베이스 마감 옵션을 선택할 수 있습니다. 제품에는 플라스틱 뒷면과 앞면의 패브릭 장식이 있는 옵션(SWC-100)과 전체 장식 옵션(SWC-110)으로 10가지 패브릭 및 6가지 가죽 옵션이 있습니다. 또한 베이스 마감 옵션으로는 스테인리스 스틸, 매트 블랙, 광택 화이트, 크롬이 있습니다. 의자에는 팔걸이가 있는 버전과 없는 버전이 있으며, 가정이나 사업장에 적합하며 계약용으로 인증되었습니다.

이 제품의 구조는 5개 바퀴가 있는 플라스틱 코팅 알루미늄 베이스로 구성되어 있습니다. 또한 손쉬운 상하 조정을 위한 공압 의자 조절 기능이 있습니다.

제품의 치수는 폭 53 CM, 깊이 51 CM, 높이 80 CM, 좌석 높이 44 CM, 좌석 깊이 41 CM입니다.

옵션으로는 부드러운 바닥 또는 단단한 바닥에 사용할 수 있는 캐스터 옵션이 있습니다. 또한 좌석 폼 밀도는 중간 (1.8 lb/ft3) 또는 높음 (2.8 lb/ft3) 중에서 선택할 수 있으며, 팔걸이는 없거나 8가지 위치 조정이 가능한 PU 팔걸이가 있습니다.

이 제품은 쉘 베이스 글라이더에 수정된 나일론 PA6/PA66 코팅이 있는 주조 알루미늄으로 제작되었습니다. 쉘의 두께는 10 mm이며, 좌석은 HD36 폼으로 만들어졌습니다.

이 제품은 이탈리아에서 제조되었습니다.


## 문제 1: 텍스트가 너무 길다

- 단어/문장/문자 수를 제한하세요.


In [37]:
# 중요한 내용은 뒤로 빼 주거나(1순위) 앞으로 뺀다(2순위). LLM은 중간을 잘 기억하지 못하는 특징이 있다.
prompt = f"""
마케팅 팀을 도와 기술 사양서를 기반으로 소매 웹사이트에 대한 제품 설명을 만듭니다.

세 개의 백틱으로 구분된 기술 사양에 제공된 정보를 기반으로 제품 설명을 작성하세요.

기술 사양: ```{fact_sheet_chair}```

최대 50단어를 사용하세요.
"""
response = get_completion(prompt)
print(response)

이 아름다운 중세 현대적인 사무용 가구 시리즈는 파일 캐비닛, 책상, 책장, 회의용 테이블 등 다양한 제품으로 구성되어 있습니다. 다양한 쉘 색상과 베이스 마감 옵션 중 선택할 수 있으며, 패브릭이 장식된 옵션과 가죽이 장식된 옵션도 있습니다. 플라스틱 코팅된 알루미늄 베이스와 상하 조절 기능이 있는 공압 의자 조절 기능이 특징입니다. 이 제품은 가정이나 사업장에 적합하며, 계약용으로 인증되었습니다.


## 문제 2. 텍스트가 잘못된 세부사항에 초점을 맞춤

- 의도한 대상에게 관련 있는 측면에 초점을 맞추도록 요청합니다.

In [38]:
prompt = f"""
마케팅 팀을 도와 기술 사양서를 기반으로 소매 웹사이트에 대한 제품 설명을 만듭니다.

세 개의 백틱으로 구분된 기술 사양에 제공된 정보를 기반으로 제품 설명을 작성하세요.

이 설명은 가구 소매업체를 대상으로 하므로, 기술적 성격을 띄어야 하며 제품이 구성된 재료에 초점을 맞춰야 합니다.

최대 50단어를 사용하세요.

기술 사양: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

새로운 중세 현대적인 사무용 가구 시리즈로, 파일 캐비닛, 책상, 책장, 회의용 테이블 등이 포함되어 있습니다. 다양한 쉘 색상과 베이스 마감 옵션 중 선택할 수 있습니다. 패브릭 장식이 있는 옵션(SWC-100)과 전체 장식 옵션(SWC-110)으로 선택할 수 있으며, 10가지 패브릭과 6가지 가죽 옵션을 제공합니다. 베이스 마감 옵션에는 스테인리스 스틸, 매트 블랙, 광택 화이트, 크롬이 있습니다. 의자는 팔걸이가 있는 버전과 없는 버전이 있으며, 가정이나 사업장에 적합합니다. 이 제품은 계약용으로 인증되었습니다. 구조는 5개 바퀴가 있는 플라스틱 코팅 알루미늄 베이스로 이루어져 있으며, 공압 의자 조절 기능을 통해 손쉬운 상하 조정이 가능합니다. 치수는 폭 53 CM, 깊이 51 CM, 높이 80 CM, 좌석 높이 44 CM, 좌석 깊이 41 CM입니다. 옵션으로는 부드러운 바닥 또는 단단한 바닥 캐스터 옵션, 중간 또는 높은 폼 밀도 선택, 팔걸이 없거나 8가지 위치 조정이 가능한 PU 팔걸이가 있습니다. 이 제품은 수정된 나일론 PA6/PA66 코팅이 있는 주조 알루미늄 쉘 베이스 글라이더와 HD36 폼으로 만들어졌으며, 이탈리아에서 제조되었습니다.


In [39]:
prompt = f"""
마케팅 팀을 도와 기술 사양서를 기반으로 소매 웹사이트에 대한 제품 설명을 만듭니다.

세 개의 백틱으로 구분된 기술 사양에 제공된 정보를 기반으로 제품 설명을 작성하세요.

이 설명은 가구 소매업체를 대상으로 하므로, 기술적 성격을 띄어야 하며 제품이 구성된 재료에 초점을 맞춰야 합니다.

설명의 마지막에 기술 사양에서 모든 7자리 제품 ID를 포함하세요.

최대 50단어를 사용하세요.

기술 사양: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

우리의 중세 현대적인 사무용 가구는 파일 캐비닛, 책상, 책장, 회의용 테이블 등을 포함한 아름다운 가족으로 구성되어 있습니다. 쉘 색상과 베이스 마감 옵션은 다양하며, 선택에 따라 플라스틱 뒷면과 앞면의 패브릭 장식이 있는 SWC-100 옵션 또는 전체 장식 옵션인 SWC-110 옵션을 선택할 수 있습니다. SWC-110 옵션에는 10가지 패브릭 및 6가지 가죽 옵션이 있습니다. 또한, 베이스 마감 옵션으로 스테인리스 스틸, 매트 블랙, 광택 화이트, 크롬을 선택할 수 있습니다. 의자에는 팔걸이가 있는 버전과 없는 버전이 있으며, 가정이나 사업장에 모두 적합합니다. 이 제품은 계약용으로 인증되었으며, 모든 7자리 제품 ID는 기술 사양에 포함되어 있습니다. 이 제품은 플라스틱 코팅된 알루미늄 베이스에 5개의 바퀴가 있으며, 공압 의자 조절 기능을 통해 손쉬운 상하 조정이 가능합니다. 제품의 치수는 폭 53cm, 깊이 51cm, 높이 80cm, 좌석 높이 44cm, 좌석 깊이 41cm입니다. 추가 옵션으로는 부드러운 바닥 또는 단단한 바닥 캐스터 옵션, 중간 또는 높은 밀도의 좌석 폼 선택, 팔걸이 없거나 8가지 위치 조정이 가능한 PU 팔걸이가 있습니다. 이 제품은 수정된 나일론 PA6/PA66 코팅이 있는 주조 알루미늄 쉘 베이스 글라이더와 HD36 폼으로 제작되었습니다. 제조국은 이탈리아입니다. (제품 ID: [제품 ID])


## 문제 3. 설명에 치수 표가 필요

In [40]:
prompt = f"""
마케팅 팀을 도와 기술 사양서를 기반으로 소매 웹사이트에 대한 제품 설명을 만듭니다.

세 개의 백틱으로 구분된 기술 사양에 제공된 정보를 기반으로 제품 설명을 작성하세요.

이 설명은 가구 소매업체를 대상으로 하므로, 기술적 성격을 띄어야 하며 제품이 구성된 재료에 초점을 맞춰야 합니다.

설명의 마지막에 기술 사양에서 모든 7자리 제품 ID를 포함하세요.

설명 후에 제품의 치수를 나타내는 표를 포함하세요. 표에는 두 개의 열이 있어야 합니다.
첫 번째 열에는 치수의 이름을 포함시키고, 두 번째 열에는 인치 단위로 측정값을 포함시키세요.

표에 'Product Dimensions'라는 제목을 부여하세요.

모든 것을 웹사이트에서 사용할 수 있는 HTML 형식으로 작성하세요.
설명을 <div> 요소 안에 배치하세요.

기술 사양: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <h2>제품 설명</h2>
  <p>
    이 가구는 중세 현대적인 사무용 가구의 아름다운 가족 중 하나로, 파일 캐비닛, 책상, 책장, 회의용 테이블 등이 포함되어 있습니다. 쉘 색상과 베이스 마감 옵션이 다양하며, 플라스틱 뒷면과 앞면의 패브릭 장식이 있는 옵션(SWC-100)과 10가지 패브릭 및 6가지 가죽 옵션이 있는 전체 장식 옵션(SWC-110)이 있습니다. 베이스 마감 옵션에는 스테인리스 스틸, 매트 블랙, 광택 화이트, 크롬이 있습니다. 의자는 팔걸이가 있는 버전과 없는 버전이 있습니다. 이 제품은 가정이나 사업장에 적합하며, 계약용으로 인증되었습니다.
  </p>
  <p>
    이 가구는 5개 바퀴가 있는 플라스틱 코팅 알루미늄 베이스를 갖추고 있습니다. 공압 의자 조절 기능으로 손쉬운 상하 조정이 가능합니다.
  </p>
  <h3>제품 치수</h3>
  <table>
    <thead>
      <tr>
        <th>치수</th>
        <th>인치</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>폭</td>
        <td>20.87"</td>
      </tr>
      <tr>
        <td>깊이</td>
        <td>20.08"</td>
      </tr>
      <tr>
        <td>높이</td>
        <td>31.50"</td>
      </tr>
      <tr>
        <td>좌석 높이</td>
        <td>17.32"</td>
      </tr>
      <tr>
        <td>좌석 깊이</td>
        <td>16.14"</td>
      </tr>
    </tbody>
  </table>
  <h3>옵션</h3>
  <ul>
    <li>부드러운 바닥 또는 단단한 바닥 캐스터 옵션</li>
    <li>좌석 폼 밀도 두 가

#### (참고) HTML 볼 수 있는 라이브러리

In [41]:
from IPython.display import display, HTML
display(HTML(response))

치수,인치
폭,"20.87"""
깊이,"20.08"""
높이,"31.50"""
좌석 높이,"17.32"""
좌석 깊이,"16.14"""


## 3. 문서 요약하기

In [42]:
prod_review = """
내 딸의 생일 선물로 이 판다 인형을 받았습니다. \
내 딸은 이 인형을 사랑하고 어디든지 가지고 다닙니다. \
부드럽고 귀여운데, 얼굴이 친근한 느낌입니다. 하지만 \
내가 지불한 가격에 비해 조금 작아 보입니다. \
같은 가격으로 더 큰 옵션이 있을 것 같아요. 예상보다 \
하루 일찍 도착해서 딸에게 주기 전에 스스로 먼저 놀 수 있었습니다.
"""

## 단어, 문장, 글자수 제한을 두고 요약하기

In [43]:
prompt = f"""
제공한 e-commerce 사이트의 제품 리뷰를 간단하게 요약하는 것이 \
당신의 과제입니다.

다음의 리뷰를, 삼중 역따옴표로 묶인 부분을 기준으로, \
최대 30단어로 요약해보세요.

리뷰: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

"내 딸을 위한 생일 선물로 받은 이 판다 인형은 부드럽고 귀여워서 딸이 너무 좋아합니다. 가격에 비해 조금 작아 보이지만, 더 큰 옵션이 있다면 더 좋을 것 같습니다. 예상보다 일찍 도착해서 딸이 놀기 전에 스스로 놀 수 있었습니다."


### "배송 및 납품"에 중점을 둔 요약

In [44]:
prompt = f"""
배송 부서에 피드백을 제공하기 위해 e-commerce 사이트의 제품 리뷰를 간단하게 요약하는 것이 \
당신의 과제입니다.

다음의 리뷰를, 삼중 역따옴표로 묶인 부분을 기준으로, \
최대 30단어로 요약하되, 제품의 배송 및 납품에 관한 어떤 측면을 언급하는지에 중점을 두세요.

리뷰: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

배송은 빠르고, 딸이 기다리지 않고 먼저 놀 수 있었으며, 판다 인형은 부드럽고 귀엽지만 크기가 작아 보입니다. 더 큰 옵션이 있으면 좋겠다.


### "가격 및 가치"에 중점을 둔 요약


In [45]:
prompt = f"""
제품의 가격을 결정하는 가격 부서에 피드백을 제공하기 위해 e-commerce 사이트의 제품 리뷰를 간단하게 요약하는 것이 \
당신의 과제입니다.

다음의 리뷰를, 삼중 역따옴표로 묶인 부분을 기준으로, \
최대 30단어로 요약하되, 제품의 가격 및 인식된 가치와 관련된 어떤 측면에 중점을 두세요.

리뷰: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

30-Word Summary: "My daughter loves this panda doll as a birthday gift, but it seems a bit small for the price. It's soft, cute, and has a friendly face. It arrived a day early, allowing me to play with it before giving it to my daughter."



### 주의:

- 요약은 주어진 리뷰에서 해당 주제에 중점을 두지만, 다른 주제와 관련된 내용도 일부 포함할 수 있습니다.

## 요약("summarize") 대신에 추출("extract")을 시도해보자.

## 배송 및 납품과 관련된 정보를 추출

In [46]:
prompt = f"""
배송 부서에 피드백을 제공하기 위해 e-commerce 사이트의 제품 리뷰에서 \
배송 및 납품과 관련된 정보를 추출하는 것이 당신의 과제입니다.

삼중 역따옴표로 묶인 부분에서 배송 및 납품과 관련된 정보를 추출하되, \
최대 30단어로 제한하세요.

리뷰: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)

배송 및 납품과 관련된 정보 추출:

- "예상보다 하루 일찍 도착"


### 여러 제품 리뷰 요약


In [48]:
review_1 = prod_review

# 무선 램프에 대한 리뷰
review_2 = """
침실용으로 좋은 램프가 필요했는데, 이 제품은 추가 저장 공간이 있고 가격도 \
높지 않아 선택했습니다. 빠르게 받았습니다 - 2일 내에 도착했습니다. \
램프의 스트링이 운송 중에 끊어졌고 회사는 기꺼이 새로운 것을 보내주었습니다. \
새로운 것 역시 몇 일 안에 도착했습니다. 램프 조립은 쉬웠습니다. 그런 다음 \
부품이 누락되었는데, 지원팀에 연락하니 빠르게 누락된 부분을 받았습니다! \
이 회사는 고객과 제품에 관심을 가지는 것 같습니다.
"""
# 전기 칫솔에 대한 리뷰
review_3 = """
내 치과 위생사는 전기 칫솔을 추천했기 때문에 이것을 구입했습니다. \
배터리 수명은 현재로서는 꽤 훌륭한 것 같습니다. 초기 충전 후에 \
배터리를 형성하기 위해 첫 주에 충전기를 연결한 후, 마지막 3주 동안 \
똑같은 충전으로 매일 두 번 브러싱했습니다. 그러나 칫솔 머리가 너무 작습니다. \
이 머리보다 큰 아기 칫솔을 본 적이 있습니다. 이 머리가 이를 통해 치아 사이에 \
더 잘 들어가게 하려면 다른 길이의 솔로 필요합니다. 전체적으로 $50 정도에 \
구할 수 있다면 좋은 거래입니다. 제조사의 교체 헤드는 꽤 비싸지만, \
더 합리적인 가격에 제공되는 일반 헤드를 구할 수 있습니다. 이 칫솔은 매일 \
치과의사에 다녀온 것 같은 느낌을 줍니다. 내 이가 반짝반짝합니다!
"""
# 믹서기에 대한 리뷰
review_4 = """
11월에 17개 조각 시스템을 약 $49에 구매할 수 있었지만, 12월의 두 번째 주에 \
동일한 시스템에 대한 가격이 (가격 인상이라고 부르겠습니다) 어떤 이유로든 $70-$89 \
사이로 상승했습니다. 그리고 11개 조각 시스템도 이전에 $29에 판매된 가격보다 \
약 $10 정도 오른 가격이었습니다. 따라보면 괜찮아 보이지만, 베이스를 보면 블레이드가 \
이전 버전에서 좋아 보이지 않습니다. 하지만 나는 이를 매우 조심스럽게 다루기 \
계획이다 (예: 빈, 얼음, 쌀 등을 블렌더에서 깨끗하게 분쇄한 다음 블렌더에서 \
원하는 서빙 사이즈로 분쇄하기 위해 처음에는 블렌더에서 크러싱, \
스무디를 만들 때 처음에 교차 컷팅 블레이드를 사용한 다음 필요하면 \
더 가는/적게 과립화된 서브 프로세서를 사용한 다음 교차 컷팅 블레이드를 \
사용하십시오. 스무디를 만들 때 팁: 과일과 야채를 미세하게 썰고 얼려두세요 \
(시금치를 사용한다면 시금치를 가볍게 데쳐서 사용할 때까지 얼려두세요 - \
소프트하게 데치면 얼려둔 다음 사용할 준비가 될 때까지, \
소르베를 만들 때 소형 또는 중형 식품 프로세서 사용) - \
이렇게 하면 스무디를 만들 때 얼음을 그다지 많이 추가하지 않아도 됩니다. \
약 1년 후에 모터가 이상한 소리를 내기 시작했습니다. 고객 서비스에 전화했지만 \
보증이 이미 만료되었으므로 새로 사야 했습니다. 참고로 이러한 제품 유형의 \
전반적인 품질이 저하되었으므로 이러한 제품의 판매를 유지하기 위해 브랜드 인식 \
및 소비자 충성도를 양산하고 있습니다. 약 2일 안에 받았습니다.
"""

In [49]:
reviews = [review_1, review_2, review_3, review_4]

In [50]:
for i in range(len(reviews)):
    prompt = f"""
    e-commerce 사이트의 제품 리뷰를 간단하게 요약하는 것이 \
    당신의 과제입니다.

    삼중 역따옴표로 묶인 부분을 기준으로, 최대 20단어로 요약하세요.

    리뷰: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 요약: "딸의 생일 선물로 받은 판다 인형은 부드럽고 귀엽지만 크기가 작아 보여요. 가격에 비해 더 큰 옵션이 있으면 좋겠어요. 도착이 예상보다 일찍해서 딸이 놀기 전에 미리 즐길 수 있었어요." 

1 침실용 램프 선택 이유: 추가 저장 공간, 저렴한 가격, 빠른 배송, 친절한 고객 서비스. 

2 전기 칫솔 구입 후 배터리 수명은 좋으나 머리가 작아 다른 솔 필요. $50 정도면 좋은 거래. 일반 헤드 구매 가능. 이 칫솔은 치과 다녀온 느낌, 이가 반짝반짝. 

3 11월에 저렴한 가격으로 17개 조각 시스템을 구매했지만, 12월에 가격이 상승하고 블레이드의 품질도 좋지 않았습니다. 스무디를 만드는데는 유용하며, 약 1년 후에 모터가 소리를 내기 시작하여 새로 구매해야 했습니다. 전반적으로 이 제품 유형의 품질이 하락하고 있습니다. 



## 3. 추론하기

- 제품 리뷰와 뉴스 기사에서 감정과 주제를 추론

## 제품 리뷰 텍스트

In [51]:
lamp_review = """
내 침실을 위한 좋은 램프가 필요했는데, 이 제품은 추가 저장 공간이 있고 가격도 \
너무 높지 않았습니다. 빨리 받았습니다. 램프 스트링이 운송 중에 끊어졌고 \
회사는 기꺼이 새로운 것을 보내주었습니다. 몇 일 안에 도착했습니다. \
조립은 쉬웠습니다. 부품이 누락되었는데, 지원팀에 연락하여 \
빠르게 누락 부분을 받았습니다! Lumina는 고객과 제품에 관심을 가지는 \
훌륭한 회사로 보입니다!!
"""

## 감정 (긍정적/부정적)

In [52]:
prompt = f"""
다음 제품 리뷰의 감정은 무엇인가요?
삼중 역따옴표로 구분된 다음 제품 리뷰의 감정은 무엇인가요?

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

이 리뷰의 감정은 긍정적입니다.


In [53]:
prompt = f"""
다음 제품 리뷰의 감정은 무엇인가요?
다음 제품 리뷰의 감정을 한 단어로 나타내주세요. "긍정적" 또는 "부정적" 중 하나로 답하세요.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

긍정적


## 감정 유형 식별

In [54]:
prompt = f"""
다음 리뷰 작성자가 표현한 감정 유형을 나열하세요. \
목록에는 다섯 개 이상의 항목을 포함하지 마세요. \
소문자 단어로 된 목록 형식으로 답하세요. 각 항목은 쉼표로 구분합니다.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

만족, 놀람, 신뢰, 감사, 칭찬


## 분노 식별


In [55]:
prompt = f"""
다음 리뷰 작성자가 분노를 표현하고 있는지 여부를 판단하세요. \
리뷰는 삼중 역따옴표로 구분됩니다. \
답변은 "예" 또는 "아니요"로 제공하세요.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

아니요


## 고객 리뷰에서 제품 및 회사 이름 추출


In [56]:
prompt = f"""
다음 리뷰 텍스트에서 다음 항목을 식별하세요:
- 리뷰어가 구매한 상품
- 해당 상품을 제조한 회사

리뷰는 삼중 역따옴표로 구분됩니다. \
"Item"과 "Brand"가 키인 JSON 객체 형식으로 \
응답을 작성하세요. 정보가 없는 경우 "unknown"을 \
값으로 사용하세요.
응답을 최대한 간결하게 작성하세요.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Item": "램프",
  "Brand": "Lumina"
}


## 여러 작업 동시에 수행

In [57]:
prompt = f"""
다음 리뷰 텍스트에서 다음 항목을 식별하세요:
- 감정 (긍정적 또는 부정적)
- 리뷰어가 분노를 표현하고 있는지 여부 (참 또는 거짓)
- 리뷰어가 구매한 상품
- 해당 상품을 제조한 회사

리뷰는 삼중 역따옴표로 구분됩니다. \
"Sentiment", "Anger", "Item" 및 "Brand"가 키인 \
JSON 객체 형식으로 응답을 작성하세요. 정보가 없는 경우 \
값으로 "unknown"을 사용하세요.
응답을 최대한 간결하게 작성하세요.
Anger 값을 부울(boolean) 형식으로 포맷하세요.

리뷰 텍스트: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "긍정적",
  "Anger": false,
  "Item": "램프",
  "Brand": "Lumina"
}


## 아까 만들었던 리뷰들을 긍정/부정에 기반하여 5점 만점의 점수로 만들어 변환하여 응답하는 프롬프트를 작성해 보세요.

In [66]:
reviews = [review_1, review_2, review_3, review_4]
# YOUR PROMPT HERE.

for i in range(len(reviews)):
    prompt = f"""
    당신이 해야 할 일은 5점 척도에 따라 리뷰에서 느껴지는 만족도를 점수화하는 것입니다.
    삼중 역따옴표로 묶인 리뷰를 읽은 뒤, 1~5점 구간의 정수로 점수를 매겨 주세요.
    각 리뷰에 대한 추가적인 언급은 제외하고, "리뷰 점수: n"의 형태로 점수만 매겨 주세요.
    리뷰: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")

0 리뷰 점수: 4 

1 리뷰 점수: 5 

2 리뷰 점수: 3 

3 리뷰 점수: 2 



## 주제 추론

In [68]:
story = """
정부가 최근 실시한 조사에서 공공부문 직원들에게 부서에서의 만족도를 평가하도록 했습니다.
결과에서는 NASA가 95%의 만족도 평가로 가장 인기 있는 부서임을 밝혀냈습니다.

NASA 직원인 존 스미스는 결과에 대해 다음과 같이 언급하며 말했습니다.
"NASA가 선두에 올 줄은 놀랍지 않아요. 그곳은 놀라운 사람들과 놀라운 기회가 있는
일하기 좋은 장소입니다. 이런 혁신적인 조직의 일원으로 자랑스러워요."

결과는 또한 NASA 경영진에 의해 환영받았습니다. 톰 존슨 이사는 다음과 같이 말했습니다.
"우리 직원들이 NASA에서의 업무에 만족하고 있다는 것을 들어서 기쁩니다.
우리는 우리의 목표를 달성하기 위해 헌신적으로 일하는 재능 있는 팀을 보유하고 있으며,
그들의 노력이 빛을 발하는 것을 보는 것은 환상적입니다."

조사는 또한 사회보장청이 가장 낮은 만족도 평가를 받았음을 밝혀냈는데,
직원의 45%만이 자신의 직장에 만족한다고 했습니다.
정부는 조사에서 직원들이 제기한 우려사항을 해결하고 모든 부서에서 직장 만족도를
향상시키기 위해 노력하기로 약속했습니다.
"""

## 5개 주제 추론

In [69]:
prompt = f"""
다음 텍스트에서 논의되고 있는 다섯 가지 주제를 결정하세요.
텍스트는 삼중 역따옴표로 구분됩니다.

각 항목을 한 단어 또는 두 단어로 작성하세요.

답변을 쉼표로 구분된 항목 목록 형식으로 작성하세요.

텍스트 샘플: '''{story}'''
"""
response = get_completion(prompt)
print(response)

조사, 공공부문, 만족도, NASA, 사회보장청


In [70]:
response.split(sep=',')

['조사', ' 공공부문', ' 만족도', ' NASA', ' 사회보장청']

## 특정 주제에 대한 뉴스 알림 생성

In [71]:
topic_list = [
    "nasa", "지방 정부", "공학",
    "직원 만족도", "연방 정부"
]
prompt = f"""
다음 목록의 각 항목이 다음 텍스트에서 주제인지 여부를 결정하세요.
텍스트는 삼중 역따옴표로 구분됩니다.

각 주제에 대해 0 또는 1로 답하세요.

주제 목록: {", ".join(topic_list)}

텍스트 샘플: '''{story}'''
"""
response = get_completion(prompt)
print(response)

nasa: 1
지방 정부: 0
공학: 0
직원 만족도: 1
연방 정부: 1


In [72]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!
